In [3]:
%load_ext autoreload
%autoreload 2

from util_0703 import *

- 실험해볼 내용들

1. 3중 for문으로 3번들을 묶으면 어떨까?
2. 좌표간 거리 계산에 유클리드 거리 대신에 민코프스키 거리를 사용하면 어떨까?
3. 초기 최적화 종료 후 인접한 번들끼리 원소를 교환해보면 어떨까?

In [4]:
problem_file = '../alg_test_problems_20240429/TEST_K100_1.json'
# problem_file = '../alg_test_problems_20240429/TEST_K100_2.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_18.json'

## 3중 for문으로 3번들을 우선적으로 묶기

In [5]:
## ------------------- 초기 상태 할당 코드 -------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

is_used = [False] * K
all_bundles = []
for i in range(K):
    if is_used[i]:
        continue
    for j in range(i + 1, K):
        if is_used[i] or is_used[j]:
            continue
        for k in range(j + 1, K):
            if is_used[i] or is_used[j] or is_used[k]:
                continue
            
            new_bundle = try_feasible_route_and_rider(K, DIST, ALL_ORDERS, ALL_RIDERS, [i, j, k])

            if new_bundle is not None:
                is_used[i] = is_used[j] = is_used[k] = True

                all_bundles.append(new_bundle)

                new_bundle.rider.available_number -= 1
                
for i in range(K):
    if is_used[i]:
        continue
    for j in range(j + 1, K):
        if is_used[i] or is_used[j]:
            continue          

        new_bundle = try_feasible_route_and_rider(K, DIST, ALL_ORDERS, ALL_RIDERS, [i, j])

        if new_bundle is not None:
            is_used[i] = is_used[j] = True

            all_bundles.append(new_bundle)

            new_bundle.rider.available_number -= 1

for i in range(K):
    if is_used[i]:
        continue

    new_bundle = try_feasible_route_and_rider(K, DIST, ALL_ORDERS, ALL_RIDERS, [i])

    if new_bundle is not None:
        is_used[i] = True

        all_bundles.append(new_bundle)

        new_bundle.rider.available_number -= 1    


In [64]:
# ## ----------------- 커스텀 배달원 재할당 코드 -----------------------

all_bundles, rider_availables = check_reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, all_bundles)
for rider_i in range(3):
    ALL_RIDERS[rider_i].available_number = rider_availables[rider_i]

## -------------- 솔루션 제작 및 실현 가능성 확인 코드 ---------------- 

solution = [
        # rider type, shop_seq, dlv_seq
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
]

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

checked_solution = solution_check(K, ALL_ORDERS, ALL_RIDERS, DIST, solution)

print(checked_solution['avg_cost'])


4175.757999999999


> 3, 2개 순으로 다중 반복문을 이용해서 묶은 결과 비용이 많이 낮아지긴 했지만 크루스칼 방식에 비해서 더 좋은 결과가 나오지는 않았다.